# Evaluate APIs, define DAO membership
- Who is a DAO member? Followers in Snapshot? Token holders? DeepDAO (active?) membership?
- In the broadest sense, DAO membership can include followers, who are also important community participants in a DAO 

In [ ]:
from util import snapshot_api
import pandas as pd

# Members, wealth, and voting
- Pick a small (<1k), medium (<10k), and large DAO (10k+)

In [ ]:
query = """
query {
  spaces(
    first: 10000
    orderBy: "followersCount"
    orderDirection: asc
  ) {
    id
    name
    followersCount
  }
}
"""

res = snapshot_api(query)
df_spaces = pd.DataFrame(res["spaces"]).sort_values("followersCount", ascending=False)
df_spaces

In [ ]:
dao_small = df_spaces.query("250 < followersCount < 750").iloc[0]
dao_medium = df_spaces.query("1000 < followersCount < 2500").iloc[0]
dao_large = df_spaces.query("10000 < followersCount < 12000").iloc[0]
print(
    f"""
SMALL: {dao_small.id} ({dao_small.followersCount:,d}) 
MEDIUM: {dao_medium.id} ({dao_medium.followersCount:,d}) 
LARGE: {dao_large.id} ({dao_large.followersCount:,d}) 
"""
)

- Get all members (if possible try different APIs)

In [ ]:
def get_followers(space):
    followers = []
    i = 0
    while True:
        skip = (i) * 1000
        first = (i + 1) * 1000
        query = (
            """
        query {
          follows(
            first: """
            + str(first)
            + """,
            skip: """
            + str(skip)
            + """,
            where: {space: """
            + f'"{space}"'
            + """}
          ){
            id
          }
        }
        """
        )
        res = snapshot_api(query)
        followers += res["follows"]
        if len(res["follows"]) < 1000:
            break
        i += 1

    df_followers = pd.DataFrame(followers)
    return df_followers

In [ ]:
df_followers_small = get_followers(dao_small.id)
df_followers_medium = get_followers(dao_medium.id)
df_followers_large = get_followers(dao_large.id)

In [ ]:
df_followers_small.info()

In [ ]:
df_followers_medium.info()

In [ ]:
df_followers_large.info()

- Order them by wealth in ETH

In [ ]:
df_followers_small.head()

- Compute correlation ETH wealth and number of votes in the DAO